In [20]:
import numpy as np
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
df = pd.read_csv('train_spam.csv')
df_test = pd.read_csv('test_spam.csv')

df.head()

,text_type,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...


In [14]:
df.shape

(16278, 2)

Закодируем целевую переменную

In [84]:
df['text_type'].unique()

array([1, 0], dtype=object)

In [9]:
df.loc[df['text_type'] == 'ham', 'text_type'] = 1
df.loc[df['text_type'] == 'spam', 'text_type'] = 0

In [10]:
df.head()

,text_type,text
0,1,make sure alex knows his birthday is over in f...
1,1,a resume for john lavorato thanks vince i will...
2,0,plzz visit my website moviesgodml to get all m...
3,0,urgent your mobile number has been awarded wit...
4,1,overview of hr associates analyst project per ...


In [35]:
print(f"text positive: {df['text_type'].sum()}")
print(f"text negative: {df.shape[0] - df['text_type'].sum()}")

text positive: 11469
text negative: 4809


В датасете есть дисбаланс классов. Для того, чтобы избежать искажения результатов, будем использовать stratify при train_test_split и метрику auc roc

In [13]:
df['text'][2]

'plzz visit my website moviesgodml to get all movies for free and also i provide direct download links no redirect and ads😊😊😊😊😁'

In [18]:
print(f"text nan: {df['text'].isna().any()}")
print(f"text_type nan: {df['text_type'].isna().any()}")

text nan: False
text_type nan: False


В тексте нет пропусков

Посмотрим на самые частые слова

In [24]:
from collections import Counter

text = ''.join(df['text']).lower()
d = Counter(text.split())

sorted_tokens = sorted(d.items(), key=lambda x: x[1], reverse=True)
print(len(d))
sorted_tokens[:10]

69567


[('the', 26800),
 ('to', 24790),
 ('and', 14857),
 ('i', 14522),
 ('a', 14000),
 ('you', 12871),
 ('1635465', 12529),
 ('of', 12174),
 ('in', 10111),
 ('for', 10003)]

Уберем мусор и посмотрим еще раз

In [25]:
import nltk
from nltk.corpus import stopwords

# -- YOUR CODE HERE --
nltk.download("stopwords", quiet=True)

filtered_tokens = [word for word in sorted_tokens if word[0] not in stopwords.words("english")]
print(len(filtered_tokens))
filtered_tokens[:10]

69416


[('1635465', 12529),
 ('ect', 5333),
 ('enron', 4753),
 ('vince', 4383),
 ('url', 2827),
 ('hou', 2658),
 ('kaminski', 2336),
 ('please', 2280),
 ('2000', 2233),
 ('com', 2213)]

В тексте есть цифры, возможно, что это будет одним из важнейших признаков спама (номер карты для скам-перевода и тп)

Разделим тренирововчную выборку

In [37]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['text_type'], test_size=0.3, stratify=df['text_type'])

In [50]:
y_train, y_test = y_train.astype('int'), y_test.astype('int')

In [62]:
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import WhitespaceTokenizer

def custom_stem_tokenizer(text):
  stemmer = SnowballStemmer("english")
  text = text.lower()
  stop = set(punctuation).union(set(stopwords.words("english")))
  tokens = [
      stemmer.stem(word)
      for word in WhitespaceTokenizer().tokenize(text)
      #if word not in stop and
      #if (len(word) != 1 or ord(word) < 128)
  ]

  return tokens

In [63]:
vectorizer = CountVectorizer(tokenizer=custom_stem_tokenizer, min_df = 0.0005)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [59]:
X_train_vec.shape

(11394, 5682)

Воспользуемся простой логистической регрессией

In [64]:
lr = LogisticRegression(max_iter=200)

lr.fit(X_train_vec, y_train)

y_pred = lr.predict(X_test_vec)

print(f"train accuracy: {accuracy_score(lr.predict(X_train_vec), y_train)}")
print(f"test accuracy: {accuracy_score(y_pred, y_test)}")

train accuracy: 0.9868351764086362
test accuracy: 0.9486076986076986


In [65]:
from sklearn.metrics import roc_auc_score

print(f"train accuracy: {roc_auc_score(lr.predict(X_train_vec), y_train)}")
print(f"test accuracy: {roc_auc_score(y_pred, y_test)}")

train accuracy: 0.9892190947559316
test accuracy: 0.945037573313783


Лучшая точность на тесте получилась 94.5%. Осталось только обучить тестовый файл и записать его в csv.

In [69]:
test = vectorizer.transform(df_test['text'])

In [70]:
y_test_values = lr.predict(test)

In [83]:
print(y_test_values.sum())
print(y_test_values.shape[0] - y_test_values.sum())

2954
1116


In [81]:
answer = pd.DataFrame(
    {
    'score': y_test_values,
    'text': df_test['text']
    }
)

In [82]:
answer.to_csv('answer', index=False)